<a href="https://colab.research.google.com/github/Zuhra66/BETC-BatteryEducation/blob/dev-ZT/Betc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Battery Education and Training Programs Across U.S. Institutions**

**Interns:**

Zuhra Totakhail

Edward Torres

Nima Mahanloo

Advisor/ Mentor: Arijit Das

Start date:8/21/2025

End date: 9/30/2025

# **Introduction:**

This dataset provides a comprehensive overview of battery-focused and related educational programs offered by institutions across the United States. With 438 entries and 35 attributes, it captures essential details about universities, colleges, technical schools, and training providers that deliver programs in areas such as battery technology, energy systems, and advanced manufacturing. The data includes institutional characteristics (e.g., accreditation, public/private status, degree level), program details (e.g., name, length, delivery mode, prerequisites, costs, and funding avenues), and industry relevance (e.g., battery-specific offerings, STEM alignment, and prospective job opportunities).

By highlighting both academic and professional training opportunities, this dataset serves as a valuable resource for researchers, policymakers, educators, and workforce development planners seeking to understand the educational landscape driving the clean energy and electric vehicle revolution.

In [28]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import matplotlib.pyplot as plt

In [29]:

# GitHub repository: https://github.com/Zuhra66/BETC-BatteryEducation'

# URL of CSV hosted on GitHub
url = "https://raw.githubusercontent.com/Zuhra66/BETC-BatteryEducation/refs/heads/main/betc_csv.csv"

# Load CSV into a pandas DataFrame
df = pd.read_csv(url)

# Make sure all columns display in one row
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Show the first 5 rows
print(df.head())




                        INSTITUTION             WEBADDR       CITY     STATE ACCREDITED (Y/N) ACCREDITATION PUBLIC PRIVATE 4-YEAR 2 YEAR (Assoc.) GRAD TECHNICAL APPRENTICE K-12  # TOTAL PROG # STEM PROG ACADEMIC  ONLINE/IN PERSON          START DATE PROG LENGTH  # BATT SPECIFIC PROGRAMS  # BATT RELATED BATT SPECIFIC      SPECIFIC PROGRAM TYPE COURSE #                                          PROG NAME                                     PROG POC                       PROG E-MAIL    PROG PHONE                                           PROG WEB                      PROG PREREQ:                                          PROG COST                          AVENUES FOR FUNDING                                       PROG ADDRESS                  PROSPECTIVE JOBS (Rough Estimate)
0  University of Michigan-Ann Arbor  https://umich.edu/  Ann Arbor  Michigan              Yes           HLC    Yes      No    Yes             No   Yes       Yes         No   No           310         124      Yes        

In [30]:
# =========================
# 0) Load
# =========================
url = "https://raw.githubusercontent.com/Zuhra66/BETC-BatteryEducation/main/betc_csv.csv"
df = pd.read_csv(url, dtype=str)  # load everything as strings first so cleaning is predictable across columns
print(f"[LOAD] Loaded CSV from GitHub: shape={df.shape} (rows, cols)")  # quick sanity check of the dataset size

# =========================
# 1) Column-name normalization -> PascalCase
# =========================
def normalize_col_pascal(col: str) -> str:
    """Helper to convert original column names to safe PascalCase."""
    s = col.strip()  # remove leading/trailing whitespace
    s = s.replace("&", "And").replace("#", "Num").replace("/", " ").replace("-", " ")
    s = s.replace("(", " ").replace(")", " ").replace(":", "")
    s = re.sub(r"\s+", " ", s).strip()  # collapse repeated spaces
    s = re.sub(r"[^0-9a-zA-Z ]", "", s)  # drop any char not alnum/space
    words = s.split()
    return "".join(w.capitalize() for w in words)  # PascalCase

orig_cols = df.columns.tolist()  # keep the original names to report what changed
df.columns = [normalize_col_pascal(c) for c in df.columns]  # apply normalization to all column names

renamed_pairs = [(o, n) for o, n in zip(orig_cols, df.columns) if o != n]  # compute which columns actually changed
print(f"[COL NAMES] Normalized to PascalCase. Renamed {len(renamed_pairs)} columns.")  # report how many columns changed
if renamed_pairs:  # avoid printing an empty example list
    print(" Examples:", renamed_pairs[:5])  # show a few examples of old->new names



[LOAD] Loaded CSV from GitHub: shape=(438, 35) (rows, cols)
[COL NAMES] Normalized to PascalCase. Renamed 35 columns.
 Examples: [('INSTITUTION', 'Institution'), ('WEBADDR', 'Webaddr'), ('CITY', 'City'), ('STATE', 'State'), ('ACCREDITED (Y/N)', 'AccreditedYN')]


In [31]:
# =========================
# 2) Trim + standardize placeholders -> Unknown
# =========================
placeholders = {"", "nan", "NaN", "N/A", "n/a", "NA", "Unknown", "unknown", "-"}  # common empty/placeholder tokens
before_placeholder_hits = 0  # counter for how many placeholders we will convert

for c in df.columns:  # iterate all columns
    s = df[c].astype(str).str.strip()  # ensure strings and trim whitespace
    before_placeholder_hits += s.isin(placeholders).sum()  # count placeholder matches before replacement

for c in df.columns:  # do the actual replacement in-place
    df[c] = df[c].astype(str).str.strip()  # normalize whitespace again to be safe
    df[c] = df[c].replace(
        {p: "Unknown" for p in placeholders}  # map all placeholder variants to "Unknown"
    )

print(f"[PLACEHOLDERS] Replaced ~{before_placeholder_hits} placeholder values with 'Unknown'.")


[PLACEHOLDERS] Replaced ~1079 placeholder values with 'Unknown'.


In [32]:
# =========================
# 3) Numeric coercion for num_* columns
# =========================
num_cols = [c for c in df.columns if c.startswith("num_")]  # choose columns intended to be numeric by naming convention
coerced_cells = 0  # track how many non-null numeric cells we end up with
for c in num_cols:  # iterate numeric columns
    before_nonnull = df[c].notna().sum()  # count non-missing cells before conversion
    df[c] = pd.to_numeric(df[c].str.replace(",", "", regex=False), errors="coerce")  # drop thousands commas and convert; invalid -> NaN
    after_numeric = df[c].notna().sum()  # count non-missing after numeric coercion
    coerced_cells += after_numeric  # accumulate non-null numeric cells
print(f"[NUMERIC] Converted {len(num_cols)} 'num_*' column(s) to numeric. Non-null numeric cells now: {coerced_cells}")  # report summary

[NUMERIC] Converted 0 'num_*' column(s) to numeric. Non-null numeric cells now: 0


In [33]:
# =========================
# 4) Boolean-like columns -> 0/1 (Int64)
# =========================
bool_cols = ["accredited","accredited_y_n","public","private","four_year",  # candidate columns that often contain yes/no values
             "two_year_assoc","grad","technical","apprentice","k_12",
             "academic","batt_specific"]
bmap = {"y":1,"yes":1,"true":1,"1":1,"t":1,"n":0,"no":0,"false":0,"0":0,"f":0}  # map several textual boolean variants to 1/0
mapped_report = []  # capture per-column mapping stats
for c in bool_cols:  # iterate possible boolean columns
    if c in df.columns:  # only process if the column exists in this dataset
        before_non_null = df[c].notna().sum()  # pre-mapping non-null count
        df[c] = df[c].str.lower().map(bmap).astype("Int64")  # lowercase -> map to 1/0 -> store as nullable integer
        after_non_null = df[c].notna().sum()  # post-mapping non-null count
        mapped_report.append((c, before_non_null, after_non_null))  # record stats for printing
print(f"[BOOLEANS] Mapped {len(mapped_report)} boolean-like column(s) to 0/1 Int64.")  # summary of how many were mapped
for c, bnn, ann in mapped_report[:8]:  # print first few columns’ stats
    print(f"  - {c}: mapped_non_null={ann} (from {bnn})")  # show how many non-null values survived mapping
if len(mapped_report) > 8:  # avoid printing too many lines
    print("  ...")  # indicate more columns were mapped

[BOOLEANS] Mapped 0 boolean-like column(s) to 0/1 Int64.


In [34]:
# =========================
# 5) Delivery mode normalization
# =========================
if "online_in_person" in df.columns:  # only derive normalized mode if source column exists
    def mode_norm(x):  # normalize to {Online, In Person, Hybrid, Unknown}
        if not isinstance(x, str): return np.nan  # non-string -> missing
        val = x.lower()  # lowercase for robust text checks
        has_online = "online" in val  # detect online
        has_person = "person" in val or "in person" in val or "on-campus" in val  # detect in-person phrasing
        if has_online and has_person: return "Hybrid"  # both present -> hybrid
        if has_online: return "Online"  # only online present
        if has_person: return "In Person"  # only in-person present
        return "Unknown"  # none of the above -> unknown
    df["delivery_mode_norm"] = df["online_in_person"].apply(mode_norm)  # create normalized delivery mode column
    counts = df["delivery_mode_norm"].value_counts(dropna=False).to_dict()  # count distribution including NaN
    print(f"[DELIVERY MODE] Added delivery_mode_norm with distribution: {counts}")  # report class balance
else:
    print("[DELIVERY MODE] Skipped (column 'online_in_person' not present).")  # note absence if not available


[DELIVERY MODE] Skipped (column 'online_in_person' not present).


In [35]:
# =========================
# 6) Parse cost to numeric (keep original text)
# =========================
if "prog_cost" in df.columns:  # only parse if a cost column exists
    def parse_cost(s: str):  # extract the first numeric value from the cost text
        if not isinstance(s, str): return np.nan  # non-string -> missing
        txt = s.strip().lower()  # normalize whitespace and case
        if txt in {"free","no cost"}: return 0.0  # treat explicit free as 0
        nums = re.findall(r"\d+(?:\.\d+)?", txt.replace(",", ""))  # find numeric tokens after removing commas
        return float(nums[0]) if nums else np.nan  # use the first number if pr

In [36]:




programs_url = "https://raw.githubusercontent.com/Zuhra66/BETC-BatteryEducation/main/betc_programs_normalized.csv"
jobs_url     = "https://raw.githubusercontent.com/Zuhra66/BETC-BatteryEducation/main/betc_jobs_normalized.csv"

programs = pd.read_csv(programs_url)
jobs     = pd.read_csv(jobs_url)

# join on record_id to see one row per program + job (left join keeps all programs)
programs_with_jobs = programs.merge(jobs, on="record_id", how="left")
print(programs_with_jobs)



      record_id                       INSTITUTION                 WEBADDR            CITY     STATE ACCREDITED (Y/N) ACCREDITATION PUBLIC PRIVATE 4-YEAR 2 YEAR (Assoc.) GRAD TECHNICAL APPRENTICE K-12  # TOTAL PROG # STEM PROG ACADEMIC  ONLINE/IN PERSON                 START DATE PROG LENGTH  # BATT SPECIFIC PROGRAMS  # BATT RELATED BATT SPECIFIC SPECIFIC PROGRAM TYPE   COURSE #                                          PROG NAME                                  PROG POC            PROG E-MAIL    PROG PHONE                                           PROG WEB                      PROG PREREQ:                                          PROG COST                          AVENUES FOR FUNDING                                       PROG ADDRESS                      job_title
0             1  University of Michigan-Ann Arbor      https://umich.edu/       Ann Arbor  Michigan              Yes           HLC    Yes      No    Yes             No   Yes       Yes         No   No           310         124 